In [1]:
from pathlib import Path
import pandas as pd

In [2]:
raw_df = pd.read_csv(
    Path("data.csv"),
    index_col="Date"
)
raw_df.head()

,Close
Date,
2021-07-19,274.766418
2021-07-20,277.057678
2021-07-21,279.120789
2021-07-22,283.822449
2021-07-23,287.323853
